# 🔬 Quantum Advantage Benchmark: Rigorous Scientific Validation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/02_quantum_advantage_benchmark.ipynb)

## Overview

This notebook implements a **publication-grade benchmarking pipeline** to rigorously test whether quantum-enhanced protein folding demonstrates measurable advantages over classical methods.

### Scientific Methodology

We employ gold-standard statistical practices:
- ✅ **Real CASP15 targets** (not synthetic data)
- ✅ **Paired comparison** (quantum vs. classical on identical data)
- ✅ **Multiple metrics** (TM-score, RMSD, GDT-TS, lDDT)
- ✅ **Statistical rigor** (Wilcoxon test, bootstrap CI, effect sizes)
- ✅ **Power analysis** (verify sufficient sample size)
- ✅ **Publication-quality figures**

### Runtime
⏱️ **30-45 minutes** on free Colab (T4 GPU)

### Output
- Comprehensive statistical analysis
- Research-grade visualizations
- LaTeX tables for papers
- Detailed results CSV

In [ ]:
# Check runtime environment
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('🚀 Running in Google Colab')
    !nvidia-smi -L
else:
    print('💻 Running locally')

## 📦 Installation & Setup

In [ ]:
if IN_COLAB:
    # Clone repository
    !git clone https://github.com/Tommaso-R-Marena/QuantumFold-Advantage.git
    %cd QuantumFold-Advantage
    
    # Install dependencies
    !pip install -q -e '.[protein-lm]'
    !pip install -q py3Dmol nglview biopython
    
    print('✅ Installation complete!')
    print('⚠️  Restarting runtime to apply numpy 2.0 upgrade...')
    print('    After restart, skip this cell and continue from imports.')
    import os
    import time
    time.sleep(2)
    os.kill(os.getpid(), 9)

In [ ]:
# Imports
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from tqdm.auto import tqdm

# QuantumFold modules
from src.advanced_model import AdvancedProteinFoldingModel
from src.protein_embeddings import ESM2Embedder
from src.benchmarks import ResearchBenchmark, StructurePredictionMetrics
from src.visualization import ProteinVisualizer
from src.data.casp_loader import CASPDataLoader

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🔧 Using device: {device}')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100
print('✅ Imports complete!')

## 📊 Load CASP15 Benchmark Dataset

We use real protein folding targets from CASP15 (Critical Assessment of protein Structure Prediction).

In [ ]:
# Load CASP15 targets
casp_loader = CASPDataLoader(casp_version=15, cache_dir='./data/casp15')

# Get diverse set of targets (varying difficulty)
targets = casp_loader.get_targets(
    max_targets=10,
    min_length=50,
    max_length=300,
    difficulty_range=['medium', 'hard']  # Focus on challenging targets
)

print(f'📁 Loaded {len(targets)} CASP15 targets')
print('\nTarget Summary:')
for i, target in enumerate(targets, 1):
    print(f"{i:2d}. {target['id']:15s} | Length: {len(target['sequence']):3d} | Difficulty: {target['difficulty']:6s}")

# Display sequence diversity
lengths = [len(t['sequence']) for t in targets]
print(f'\nLength range: {min(lengths)}-{max(lengths)} residues')
print(f'Mean length: {np.mean(lengths):.1f} ± {np.std(lengths):.1f}')

## 🧬 Initialize Models

We create **paired models** - identical architecture except for quantum enhancement.

In [ ]:
# Load ESM-2 embedder (shared by both models)
print('Loading ESM-2 embedder...')
embedder = ESM2Embedder(model_name='esm2_t33_650M_UR50D', device=device)
print('✅ ESM-2 loaded')

# Model configuration
model_config = {
    'input_dim': 1280,
    'c_s': 384,
    'c_z': 128,
    'num_encoder_layers': 8,
    'num_structure_layers': 6,
    'num_heads': 8,
}

# Quantum-enhanced model
print('\nInitializing Quantum model...')
quantum_model = AdvancedProteinFoldingModel(
    **model_config,
    use_quantum=True,
    num_qubits=8,
    quantum_depth=4
).to(device)

# Classical baseline (identical except no quantum layers)
print('Initializing Classical baseline...')
classical_model = AdvancedProteinFoldingModel(
    **model_config,
    use_quantum=False
).to(device)

# Load pretrained weights if available
quantum_checkpoint = 'outputs/quantum_model/best_model.pt'
classical_checkpoint = 'outputs/classical_model/best_model.pt'

if os.path.exists(quantum_checkpoint):
    quantum_model.load_state_dict(torch.load(quantum_checkpoint, map_location=device))
    print('✅ Loaded quantum checkpoint')
else:
    print('⚠️  No quantum checkpoint found - using random initialization')

if os.path.exists(classical_checkpoint):
    classical_model.load_state_dict(torch.load(classical_checkpoint, map_location=device))
    print('✅ Loaded classical checkpoint')
else:
    print('⚠️  No classical checkpoint found - using random initialization')

quantum_model.eval()
classical_model.eval()

# Count parameters
quantum_params = sum(p.numel() for p in quantum_model.parameters())
classical_params = sum(p.numel() for p in classical_model.parameters())
print(f'\nQuantum model: {quantum_params:,} parameters')
print(f'Classical model: {classical_params:,} parameters')

## 🎯 Run Predictions on All Targets

We predict structures using both models on identical inputs.

In [ ]:
# Storage for results
results = {
    'target_id': [],
    'sequence_length': [],
    'difficulty': [],
    'quantum': {'coords': [], 'confidence': [], 'time': []},
    'classical': {'coords': [], 'confidence': [], 'time': []},
    'true_coords': []
}

print('🚀 Running predictions...\n')

for target in tqdm(targets, desc='Targets'):
    target_id = target['id']
    sequence = target['sequence']
    true_coords = target['coordinates']  # CA coordinates
    
    # Get embeddings (shared)
    with torch.no_grad():
        embeddings = embedder([sequence])
        emb_tensor = embeddings['embeddings'].to(device)
    
    # Quantum prediction
    import time
    start_time = time.time()
    with torch.no_grad():
        quantum_output = quantum_model(emb_tensor)
    quantum_time = time.time() - start_time
    
    # Classical prediction
    start_time = time.time()
    with torch.no_grad():
        classical_output = classical_model(emb_tensor)
    classical_time = time.time() - start_time
    
    # Store results
    results['target_id'].append(target_id)
    results['sequence_length'].append(len(sequence))
    results['difficulty'].append(target['difficulty'])
    results['true_coords'].append(true_coords)
    
    results['quantum']['coords'].append(quantum_output['coordinates'].cpu().numpy()[0])
    results['quantum']['confidence'].append(quantum_output['plddt'].cpu().numpy()[0])
    results['quantum']['time'].append(quantum_time)
    
    results['classical']['coords'].append(classical_output['coordinates'].cpu().numpy()[0])
    results['classical']['confidence'].append(classical_output['plddt'].cpu().numpy()[0])
    results['classical']['time'].append(classical_time)

print('\n✅ Predictions complete!')
print(f'Average inference time - Quantum: {np.mean(results["quantum"]["time"]):.3f}s | Classical: {np.mean(results["classical"]["time"]):.3f}s')

## 📊 Compute Structural Metrics

Calculate comprehensive metrics for each prediction.

In [ ]:
# Initialize benchmark
benchmark = ResearchBenchmark(alpha=0.05, n_bootstrap=10000)

# Compute metrics for all targets
metrics_data = {
    'quantum': [],
    'classical': []
}

print('Computing structural metrics...\n')

for i in tqdm(range(len(results['target_id'])), desc='Metrics'):
    target_id = results['target_id'][i]
    true_coords = results['true_coords'][i]
    sequence = targets[i]['sequence']
    
    # Quantum metrics
    quantum_coords = results['quantum']['coords'][i]
    quantum_conf = results['quantum']['confidence'][i]
    quantum_metrics = benchmark.compute_all_metrics(
        quantum_coords, true_coords, sequence, quantum_conf
    )
    metrics_data['quantum'].append(quantum_metrics)
    
    # Classical metrics
    classical_coords = results['classical']['coords'][i]
    classical_conf = results['classical']['confidence'][i]
    classical_metrics = benchmark.compute_all_metrics(
        classical_coords, true_coords, sequence, classical_conf
    )
    metrics_data['classical'].append(classical_metrics)

print('\n✅ Metrics computed!')

# Create summary DataFrame
summary_rows = []
for i, target_id in enumerate(results['target_id']):
    row = {
        'Target': target_id,
        'Length': results['sequence_length'][i],
        'Difficulty': results['difficulty'][i],
    }
    # Add quantum metrics
    for key, val in metrics_data['quantum'][i].to_dict().items():
        row[f'Q_{key}'] = val
    # Add classical metrics
    for key, val in metrics_data['classical'][i].to_dict().items():
        row[f'C_{key}'] = val
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)
print('\nPer-Target Summary:')
display(summary_df[['Target', 'Length', 'Q_TM-score', 'C_TM-score', 'Q_RMSD (Å)', 'C_RMSD (Å)']])

## 🔬 Statistical Analysis

Rigorous hypothesis testing to determine if quantum model shows significant improvement.

In [ ]:
# Extract metric arrays
quantum_tm = np.array([m.tm_score for m in metrics_data['quantum']])
classical_tm = np.array([m.tm_score for m in metrics_data['classical']])

quantum_rmsd = np.array([m.rmsd for m in metrics_data['quantum']])
classical_rmsd = np.array([m.rmsd for m in metrics_data['classical']])

quantum_gdt = np.array([m.gdt_ts for m in metrics_data['quantum']])
classical_gdt = np.array([m.gdt_ts for m in metrics_data['classical']])

quantum_lddt = np.array([m.lddt for m in metrics_data['quantum']])
classical_lddt = np.array([m.lddt for m in metrics_data['classical']])

# Perform statistical comparisons
print('📈 Statistical Analysis\n')
print('='*80)

# TM-score comparison
tm_results = benchmark.compare_methods(
    quantum_tm, classical_tm,
    metric_name='TM-score',
    higher_is_better=True
)

print('\n1. TM-SCORE COMPARISON')
print('-' * 80)
print(f'Quantum:   {tm_results["quantum_mean"]:.4f} ± {tm_results["quantum_std"]:.4f}')
print(f'Classical: {tm_results["classical_mean"]:.4f} ± {tm_results["classical_std"]:.4f}')
print(f'Difference: {tm_results["quantum_mean"] - tm_results["classical_mean"]:.4f}')
print(f'95% CI (difference): [{tm_results["difference_ci"][0]:.4f}, {tm_results["difference_ci"][1]:.4f}]')
print(f'\nWilcoxon p-value: {tm_results["wilcoxon_pvalue"]:.6f}')
print(f'Cohen\'s d: {tm_results["cohens_d"]:.3f}')
print(f'Statistical Power: {tm_results["power"]:.3f}')
print(f'Significant at α=0.05: {"YES ✅" if tm_results["significant"] else "NO ❌"}')

# RMSD comparison (lower is better)
rmsd_results = benchmark.compare_methods(
    quantum_rmsd, classical_rmsd,
    metric_name='RMSD',
    higher_is_better=False
)

print('\n2. RMSD COMPARISON')
print('-' * 80)
print(f'Quantum:   {rmsd_results["quantum_mean"]:.4f} ± {rmsd_results["quantum_std"]:.4f} Å')
print(f'Classical: {rmsd_results["classical_mean"]:.4f} ± {rmsd_results["classical_std"]:.4f} Å')
print(f'Difference: {rmsd_results["quantum_mean"] - rmsd_results["classical_mean"]:.4f} Å')
print(f'Wilcoxon p-value: {rmsd_results["wilcoxon_pvalue"]:.6f}')
print(f'Cohen\'s d: {rmsd_results["cohens_d"]:.3f}')
print(f'Significant: {"YES ✅" if rmsd_results["significant"] else "NO ❌"}')

# GDT-TS comparison
gdt_results = benchmark.compare_methods(
    quantum_gdt, classical_gdt,
    metric_name='GDT-TS',
    higher_is_better=True
)

print('\n3. GDT-TS COMPARISON')
print('-' * 80)
print(f'Quantum:   {gdt_results["quantum_mean"]:.2f} ± {gdt_results["quantum_std"]:.2f}')
print(f'Classical: {gdt_results["classical_mean"]:.2f} ± {gdt_results["classical_std"]:.2f}')
print(f'Wilcoxon p-value: {gdt_results["wilcoxon_pvalue"]:.6f}')
print(f'Significant: {"YES ✅" if gdt_results["significant"] else "NO ❌"}')

print('\n' + '='*80)

## 📊 Publication-Quality Visualizations

In [ ]:
# TM-score comparison plot
fig = benchmark.plot_comparison(
    quantum_tm, classical_tm,
    metric_name='TM-score',
    figsize=(16, 5)
)
plt.savefig('tm_score_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# RMSD comparison plot
fig = benchmark.plot_comparison(
    quantum_rmsd, classical_rmsd,
    metric_name='RMSD (Å)',
    figsize=(16, 5)
)
plt.savefig('rmsd_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Multi-metric radar plot
from math import pi

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

metrics = ['TM-score', 'GDT-TS', 'lDDT', 'Contact\nPrecision']
quantum_vals = [
    np.mean(quantum_tm),
    np.mean(quantum_gdt) / 100,  # Normalize to 0-1
    np.mean(quantum_lddt) / 100,
    np.mean([m.contact_precision for m in metrics_data['quantum']])
]
classical_vals = [
    np.mean(classical_tm),
    np.mean(classical_gdt) / 100,
    np.mean(classical_lddt) / 100,
    np.mean([m.contact_precision for m in metrics_data['classical']])
]

angles = [n / len(metrics) * 2 * pi for n in range(len(metrics))]
quantum_vals += quantum_vals[:1]
classical_vals += classical_vals[:1]
angles += angles[:1]

ax.plot(angles, quantum_vals, 'o-', linewidth=2, label='Quantum', color='#FF6B6B')
ax.fill(angles, quantum_vals, alpha=0.25, color='#FF6B6B')
ax.plot(angles, classical_vals, 'o-', linewidth=2, label='Classical', color='#4ECDC4')
ax.fill(angles, classical_vals, alpha=0.25, color='#4ECDC4')

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics, size=12)
ax.set_ylim(0, 1)
ax.set_title('Multi-Metric Performance Comparison', size=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=12)
ax.grid(True)

plt.savefig('radar_plot.png', dpi=300, bbox_inches='tight')
plt.show()

## 📄 Generate LaTeX Tables for Publication

In [ ]:
# Generate LaTeX table
latex_table = benchmark.generate_latex_table(
    tm_results,
    caption='Quantum vs. Classical TM-score Comparison on CASP15 Targets'
)

print('\n' + '='*80)
print('LaTeX Table (copy to your paper):')
print('='*80)
print(latex_table)
print('='*80)

# Save to file
with open('results_table.tex', 'w') as f:
    f.write(latex_table)
print('\n✅ Saved to results_table.tex')

## 💾 Save Complete Results

In [ ]:
# Save detailed results
summary_df.to_csv('benchmark_results.csv', index=False)
print('✅ Saved benchmark_results.csv')

# Save statistical results
stats_summary = {
    'TM-score': tm_results,
    'RMSD': rmsd_results,
    'GDT-TS': gdt_results
}

import json
with open('statistical_results.json', 'w') as f:
    # Convert numpy types to Python types for JSON
    json_safe = {}
    for metric, results in stats_summary.items():
        json_safe[metric] = {k: float(v) if isinstance(v, (np.floating, np.integer)) else v 
                             for k, v in results.items() if k not in ['quantum_ci', 'classical_ci', 'difference_ci']}
    json.dump(json_safe, f, indent=2)

print('✅ Saved statistical_results.json')

# Download in Colab
if IN_COLAB:
    from google.colab import files
    files.download('benchmark_results.csv')
    files.download('statistical_results.json')
    files.download('results_table.tex')

## 📝 Summary & Interpretation

### Key Findings

Based on the statistical analysis above:

1. **TM-score**: Quantum model shows [FILL BASED ON RESULTS]
2. **RMSD**: [FILL BASED ON RESULTS]
3. **GDT-TS**: [FILL BASED ON RESULTS]

### Statistical Significance

- Wilcoxon signed-rank test p-value: [FILL]
- Effect size (Cohen's d): [FILL]
- Statistical power: [FILL]

### Interpretation

[FILL WITH SCIENTIFIC INTERPRETATION]

### Next Steps

- Expand to full CASP15 dataset
- Test on CASP16 targets (when available)
- Conduct ablation studies on quantum components
- Optimize hyperparameters
- Scale to larger proteins (>500 residues)